In [1]:
# index 5  depth
# index 8  feature
# index 11 auc
# index 14 validation error
# index 17 training error

data = []

file = "ddd.txt"
i = 0
for line in open(file):
    datum = line.split()
    data.append(datum)





In [4]:
rankAUC = sorted(data, key=lambda l: -float(l[11]))
print rankAUC[0]

['150000', 'Train', 'Data', 'depth', '=', '20', 'feature', '=', '1', 'ValAUC', '=', '0.718990003173', 'ValErr', '=', '0.26306', 'TraErr', '=', '0.148546666667']


In [7]:
rankTrainValDiff = sorted(data, key = lambda l:   abs(float(l[17]) - float(l[14])))
print rankTrainValDiff[0]

['150000', 'Train', 'Data', 'depth', '=', '20', 'feature', '=', '1', 'ValAUC', '=', '0.718990003173', 'ValErr', '=', '0.26306', 'TraErr', '=', '0.148546666667']


In [8]:
# init
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
import mltools as ml
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestRegressor

In [15]:
# load data
X = np.genfromtxt('data/X_train.txt', delimiter=None)
Y = np.genfromtxt('data/Y_train.txt', delimiter=None)
Xte = np.genfromtxt('data/X_test.txt', delimiter=None)

np.random.seed(0)
X,Y = ml.shuffleData(X,Y)

In [16]:
# Setting Xtr, Ytr, Xva, Yva
Xtr = X[:150000, :]
Ytr = Y[:150000]
Xva = X[150000:, :]
Yva = Y[150000:]

In [17]:
def calcError(prediction, real):
    err_count = 0.
    for i in range(len(prediction)):
        if prediction[i] != real[i]:
            err_count+=1
    return err_count / len(prediction)

In [18]:
def convert(regress_list):
    result = []
    for i in regress_list:
        if i < 0.5:
            result.append(0)
        else:
            result.append(1)
    return result

In [19]:
learner = RandomForestRegressor(max_depth=18, 
                                random_state=0, 
                                n_estimators = 500, 
                                max_features = 1)
learner.fit(Xtr, Ytr)
Yva_hat = convert(learner.predict(Xva))
validation_auc = roc_auc_score(Yva_hat, Yva)

print "Validation AUC", validation_auc
print "Validation Error", calcError(Yva_hat, Yva)
Ytr_hat = convert(learner.predict(Xtr))
print "Train Error", calcError(Ytr_hat, Ytr), "\n"

Validation AUC 0.725273555801
Validation Error 0.2647
Train Error 0.18286 



In [20]:
Yte_hat = learner.predict(Xte)
Yte = np.vstack((np.arange(Xte.shape[0]), Yte_hat)).T
np.savetxt('Y_submit_random_forestForEnsemble.txt',Yte,'%d, %.2f',header='ID,Prob1',comments='',delimiter=',')